In [4]:
import torch
from torch import nn
import torchvision.models as models
import torchvision
from revuresnet18 import revuresnet18
import warnings
warnings.filterwarnings("ignore")

In [6]:
model = models.resnet18(pretrained = True)
print(model.fc)

Linear(in_features=512, out_features=1000, bias=True)


In [ ]:
def loader(index, batch_size):
    superpath = "/home/ghostvortex/Documents/DL_Project/Dataset/"
    voxel = []
    normal_img = []
    depth_img = []
    sil_img = []
    for folder in sorted(os.listdir(superpath))[index+1:(index+int(batch_size/20)+1)]:
        for folder_in in sorted(os.listdir(superpath+folder)):
            for files in sorted(os.listdir(superpath+folder+"/"+folder_in)):
                #print(count, " ",superpath+folder+"/"+folder_in+"/"+files)
                if "rotvox" in files:
                    img = np.load(superpath+folder+"/"+folder_in+"/"+files)
                    label = np.where(labels['voxel']>0.5,1,0)
                    transform = torchvision.transforms.ToTensor()
                    img = transform(img).unsqueeze(0)
                    voxel.append(img)
                    
                elif "depth" in files:
                    img = cv2.imread(superpath+folder+"/"+folder_in+"/"+files)
                    transform = torchvision.transforms.ToTensor()
                    img = transform(img[:,:,0]).unsqueeze(0)
                    depth_img.append(img)
                elif "normal" in files:
                    img = cv2.imread(superpath+folder+"/"+folder_in+"/"+files)
                    transform = torchvision.transforms.ToTensor()
                    img = transform(img).unsqueeze(0)
                    normal_img.append(img)
                elif "sil" in files:
                    img = cv2.imread(superpath+folder+"/"+folder_in+"/"+files)
                    transform = torchvision.transforms.ToTensor()
                    img = transform(img[:,:,0]).unsqueeze(0)
                    sil_img.append(img)
                    
    voxel1 = torch.cat(voxel, dim=0)
    normal_img1 = torch.cat(normal_img, dim=0)
    depth_img1 = torch.cat(depth_img, dim=0)
    sil_img1 = torch.cat(sil_img, dim=0)
    
    is_bg = sil_img1 <= 0 #self.silhou_thres
    depth_img1[is_bg] = 0
    normal_img1[is_bg.repeat(1, 3, 1, 1)] = 0 # NOTE: if old net2, set to white (100),
    x = torch.cat((depth_img1, normal_img1), 1) # and swap depth and normal             

    return voxel1, x

In [7]:
class Model3d(torch.nn.Module):
    def __init__(self):
        
        super(Model3d, self).__init__()
        self.model = models.resnet18(pretrained = True)
        
        self.model.conv1 = nn.Conv2d(4, 64, 7, stride=2, padding=3, bias=False)
        self.model.avgpool = nn.AdaptiveAvgPool2d(1)
        self.model.fc = nn.Linear(512, 200) #encode_dim =200 (latent vec dim)
        self.encoder = nn.Sequential(self.model)
        
        n_dims=200; 
        nf=512;
        
        self.decoder = nn.Sequential(
            
        nn.ConvTranspose3d(n_dims, nf, 4, stride=1, padding=0, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),

        nn.ConvTranspose3d(nf, nf//2, 4, stride=2, padding=1, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf//2, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),

        nn.ConvTranspose3d(nf//2, nf//4, 4, stride=2, padding=1, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf//4, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),

        nn.ConvTranspose3d(nf//4, nf//8, 4, stride=2, padding=1, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf//8, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),

        nn.ConvTranspose3d(nf//8, nf//16, 4, stride=2, padding=1, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf//16, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),
        nn.ConvTranspose3d(nf//16, 1, 4, stride=2, padding=1, dilation=1, groups=1, bias=True)
        )



    def forward(self,x):
        latent_vec = self.encoder(x.float())
        latent_vec = latent_vec.view(latent_vec.size(0), -1, 1, 1, 1)
        vox = self.decoder(latent_vec)
        
        return vox;
        
    
        

In [48]:
class Trainer():
    def __init__(self,model):
        self.model = model
        
    def train(self, model):
        
        num_epochs = 500
        num_batch  = 1
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        criterion = torch.nn.BCEWithLogitsLoss(reduction = 'elementwise_mean')
        optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
        
        if torch.cuda.device_count() >= 1:
            print("Let's use", torch.cuda.device_count(), "GPUs!")
        model.to(device)
        
        f = open('/home/ghostvortex/models/status-second-module.txt','w').close()

        
        for epoch in range(num_epochs):
            ind = 0
            
            for i in range(int(2150/(batch_size/20))):
                
                print("Epoch:", epoch,", Iteration:",i)
                # Load data and labels of that batch
                f = open('/home/ghostvortex/models/status-first-module.txt','a')
                f.write("Epoch:  " + str(epoch) + ",  Iteration:" + str(i)+ "\n")
                f.close()
                
                voxel, x = loader(ind, batch_size)
                
                
                out = voxel.to(device)
                inp = x.to(device)
                
                try:
                    output = model.forward(inp)

                except RuntimeError as e:
                    if 'out of memory' in str(e):
                        sys.stdout.flush()
                        for p in model.parameters():
                            if p.grad is not None:
                                del p.grad
                        torch.cuda.empty_cache()
                        output = model.forward(inp)
                    else: raise e

                loss = criterion(output,label.type(torch.FloatTensor))      
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                ind += batch_size // 20
            
            print("Epoch: ", epoch , "Loss: ", loss.item())
            f = open('/home/ghostvortex/models/status-first-module.txt','a')
            f.write("\n"+"Epoch:  " + str(epoch) + ",  Loss:" + str(loss.item())+ "\n")
            f.close()
        f.close()
        return model
                

model = Model3d()
trainer = Trainer(model)
trained_model = trainer.train(model)
torch.save(trained_model.state_dict(),'/home/ghostvortex/models/second_module.pth')

Epoch:  0 Loss:  0.8094754219055176
Epoch:  10 Loss:  0.16951245069503784
Epoch:  20 Loss:  0.1525782346725464
Epoch:  30 Loss:  0.14245781302452087
Epoch:  40 Loss:  0.13064508140087128
Epoch:  50 Loss:  0.1194363683462143
Epoch:  60 Loss:  0.11170250922441483
Epoch:  70 Loss:  0.10146713256835938
Epoch:  80 Loss:  0.08196892589330673
Epoch:  90 Loss:  0.07366722077131271
Epoch:  100 Loss:  0.06507912278175354
Epoch:  110 Loss:  0.053190600126981735
Epoch:  120 Loss:  0.04480641335248947
Epoch:  130 Loss:  0.036886066198349
Epoch:  140 Loss:  0.031138595193624496
Epoch:  150 Loss:  0.026902638375759125
Epoch:  160 Loss:  0.023647820577025414
Epoch:  170 Loss:  0.021144459024071693
Epoch:  180 Loss:  0.019044533371925354
Epoch:  190 Loss:  0.017489340156316757
Epoch:  200 Loss:  0.015617168508470058
Epoch:  210 Loss:  0.014504157938063145
Epoch:  220 Loss:  0.013246258720755577
Epoch:  230 Loss:  0.012325410731136799
Epoch:  240 Loss:  0.011560434475541115
Epoch:  250 Loss:  0.01048067

In [94]:
# Testing
#V = trained_model.forward(x)
#V = V.detach().numpy().squeeze()
#temp = np.where(V < 0, 0, 1)

In [93]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# z,x,y = temp.nonzero()
# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(x, y, -z, zdir='z', c = 'red')
# #ax.view_init(azim=-45)
# plt.show()

In [92]:
# from mpl_toolkits.mplot3d.art3d import Poly3DCollection
# from skimage import measure
# from skimage.draw import ellipsoid
# verts, faces, normals, values = measure.marching_cubes_lewiner(temp, 0)
# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(111, projection='3d')

# # Fancy indexing: `verts[faces]` to generate a collection of triangles
# mesh111 = Poly3DCollection(verts[faces])
# mesh111.set_edgecolor('k')
# ax.add_collection3d(mesh111)

# ax.set_xlim(0, 128) 
# ax.set_ylim(0, 128)  
# ax.set_zlim(0, 128)  
# #ax.view_init(azim=-45)
# plt.tight_layout()
# plt.show()
# print(verts.shape)